In [23]:
import statistics
import queue
from collections import namedtuple

In [24]:
Item = namedtuple("Item", ['index', 'value', 'weight', 'value_per_weight'])
items = [Item(index=0, value=1, weight=2, value_per_weight=0.5), Item(index=1, value=1, weight=2, value_per_weight=0.5), Item(index=2, value=1, weight=2, value_per_weight=0.5), Item(index=3, value=13, weight=7, value_per_weight=1.8571428571428572), Item(index=4, value=10, weight=5, value_per_weight=2.0), Item(index=5, value=10, weight=5, value_per_weight=2.0), Item(index=6, value=8, weight=3, value_per_weight=2.6666666666666665)]

In [25]:
# Item = namedtuple("Item", ['index', 'value', 'weight'])
# items = [Item(index=0, value=45, weight=5), Item(index=1, value=48, weight=8), Item(index=2, value=35, weight=3)]

In [26]:
Node = namedtuple("Node", ['direction', 'index', 'branch', 'value', 'weight', 'estimate', 'excluded_paths'])

In [27]:
def get_children(node_value, capacity, node_estimate, items, branch, paths):
        
    optimistic_value = 0
    total_weight =  MAX_CAPACITY
    filtered_items = [item for item in items if item.index not in paths]
    
    for item in sorted(filtered_items, key=lambda x: x.value_per_weight, reverse=True):
        if total_weight-item.weight > 0 :
            optimistic_value += item.value
            total_weight -= item.weight  
        else:
            optimistic_value += total_weight * item.value_per_weight
            break
            
    optimistic_value = int(optimistic_value)

    item = items[branch]
    
    if capacity-item.weight >= 0 :
        left_node = Node('left', item.index, branch+1, node_value+item.value, capacity-item.weight, node_estimate, paths)
        right_node = Node('right', item.index, branch+1, node_value, capacity, optimistic_value, paths+[item.index]) #node_estimate-item.value) 
    else:
        left_node = Node('left', item.index, branch+1, -1, capacity-item.weight, -1, paths)
        right_node = Node('right', item.index, branch+1, node_value, capacity, optimistic_value, paths+[item.index]) #node_estimate-item.value)

    return left_node, right_node

<!-- ## Depth First Search -->

In [52]:
with open('./data/ks_30_0', 'r') as input_data_file:
    input_data = input_data_file.read()

# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
total_items = int(firstLine[0])
MAX_CAPACITY = int(firstLine[1])

items = []

for i in range(1, total_items+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1]),  int(parts[0])/int(parts[1])))

min_capacity = min([item.weight for item in items])
total_items = len(items)

optimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight > 0 :
        optimistic_value += item.value
        total_weight -= item.weight  
    else:
        optimistic_value += total_weight * item.value_per_weight
        break

optimistic_value = int(optimistic_value)

pessimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=False):
    if total_weight-item.weight > 0 :
        pessimistic_value += item.value
        total_weight -= item.weight  
    else:
        pessimistic_value += total_weight * item.value_per_weight
        break
        
pessimistic_value = int(pessimistic_value)     
# min_capacity = MAX_CAPACITY - total_weight + int(total_weight/item.value_per_weight)

total_weight = MAX_CAPACITY
max_items=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight > 0 :
        total_weight -= item.weight  
        max_items+=1
    else:
        break

print('max_capacity: ', MAX_CAPACITY)
print('min_capacity: ', min_capacity)
print('optimistic_value: ', optimistic_value)
print('pessimistic_value: ', pessimistic_value)
print('total_items: ', total_items)
print('max_items: ', max_items)
## 99798
## 100236

max_capacity:  100000
min_capacity:  10002
optimistic_value:  99998
pessimistic_value:  99915
total_items:  30
max_items:  9


In [53]:
statistics.median([item.weight for item in items]), statistics.mean([item.weight for item in items])

(86376.0, 52432.53333333333)

In [54]:
pessimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=False):
    if total_weight-item.weight > 0 :
        pessimistic_value += item.value
        total_weight -= item.weight  
    else:
        pessimistic_value += total_weight * item.value_per_weight
        break
        
pessimistic_value = int(pessimistic_value)     
# min_capacity = MAX_CAPACITY - total_weight + int(total_weight/item.value_per_weight)

In [55]:
items = sorted(items, key=lambda x: x.weight, reverse=True)
new_list = []
for i in range(10):
    new_list+=items[i::10]

items = new_list

In [56]:
max_value = 0
unexplored_nodes = []
optimal_path = []
optimal_node = None

unexplored_nodes = queue.PriorityQueue()

left_node, right_node = get_children(0, MAX_CAPACITY, optimistic_value, items, 0, [])

# unexplored_nodes.append(right_node)
# unexplored_nodes.append(left_node)

unexplored_nodes.put((-right_node.estimate, right_node))
unexplored_nodes.put((-left_node.estimate, left_node))

# # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.branch, x.estimate))
unexplored_nodes

In [32]:
loop_counter = 0

while not unexplored_nodes.empty(): #len(unexplored_nodes) > 0:
    
    # explore_node = unexplored_nodes.pop()
    _, explore_node = unexplored_nodes.get()

    # if explore_node.branch == len(items):
    #     continue
    
    left_node, right_node = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch, explore_node.excluded_paths)

    loop_counter += 1
    
    # print('Exploring: ', explore_node)
    # print('Max Value: ', max_value)
    # print('Left Child: ', left_node)
    # print('Right Child: ', right_node)
    
    if right_node.estimate > max_value and right_node.branch != total_items:
        # unexplored_nodes.append(right_node)   
        unexplored_nodes.put((-right_node.estimate, right_node))
    else:     
        if right_node.value > max_value:
            max_value = right_node.value
            optimal_node = right_node
            
                    
            # unexplored_nodes = [node for node in unexplored_nodes if node.estimate > max_value]
            # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.estimate, reverse=False)

    # if left_node.branch == len(items):
    #     should_expand = False
    # else:
    #     should_expand = left_node.weight+items[left_node.branch].weight >= pessimistic_value or left_node.estimate >= pessimistic_value

    if left_node.weight >= min_capacity and right_node.branch != total_items:
         # unexplored_nodes.append(left_node)
        unexplored_nodes.put((-left_node.estimate, left_node))
    else:     
        if left_node.value > max_value:
            max_value = left_node.value
            optimal_node = left_node
    
    # unexplored_nodes = [node for node in unexplored_nodes if node.estimate > max_value]
    # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.estimate, reverse=False)
    
    
            
    # print('Max Value: ', max_value)
    # print(unexplored_nodes)
    # print('---------------------------------------')

In [33]:
optimal_path = [1]*len(items)
ignore_item = False
for item in items:
    if item.index in optimal_node.excluded_paths or ignore_item:
       optimal_path[item.index] = 0

    elif item.index == optimal_node.index:
        ignore_item = True

In [34]:
max_value, loop_counter

(99798, 6651)

In [14]:
6771, 6655, 283

(6771, 6655, 283)

In [ ]:
46957
67082

In [ ]:
loop_counter = 0

while len(unexplored_nodes) > 0:
    
    loop_counter += 1
        
    explore_node = unexplored_nodes.pop()

    if explore_node.branch == len(items):
        continue
    
    left_node, right_node = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch, explore_node.excluded_paths)
    
    # print('Exploring: ', explore_node)
    # print('Max Value: ', max_value)
    # print('Left Child: ', left_node)
    # print('Right Child: ', right_node)
    
    if right_node.estimate > max_value:
        unexplored_nodes.append(right_node)   
    else:     
        if right_node.value > max_value:
            max_value = right_node.value
            optimal_node = right_node
            
                    
            # unexplored_nodes = [node for node in unexplored_nodes if node.estimate > max_value]
    

    if left_node.weight >= min_capacity and left_node.estimate >= max_value:
         unexplored_nodes.append(left_node)
    else:     
        if left_node.value > max_value:
            max_value = left_node.value
            optimal_node = left_node
    
            # unexplored_nodes = [node for node in unexplored_nodes if node.estimate > max_value]
            # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.estimate, reverse=False)
    
    # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.branch, reverse=True)
            
    # print('Max Value: ', max_value)
    # print(unexplored_nodes)
    # print('---------------------------------------')

In [ ]:
optimal_path = [1]*len(items)
ignore_item = False
for item in items:
    if item.index in optimal_node.excluded_paths or ignore_item:
       optimal_path[item.index] = 0

    elif item.index == optimal_node.index:
        ignore_item = True

In [ ]:
optimal_node

In [ ]:
max_value, loop_counter

In [39]:
def create_deciles(data):
    # Sort the data based on the numerical value in each tuple
    sorted_data = sorted(data, key=lambda x: x.weight)

    # Calculate the number of tuples in each decile
    num_tuples = len(sorted_data)
    tuples_per_decile = num_tuples // 10

    # Create deciles
    deciles = [sorted_data[i * tuples_per_decile: (i + 1) * tuples_per_decile] for i in range(10)]

    return deciles

# Example list of tuples (unique identifier, numerical value)
data = [("A", 15), ("B", 8), ("C", 22), ("D", 5), ("E", 12), ("F", 30), ("G", 18), ("H", 25), ("I", 10), ("J", 27)]
deciles = create_deciles(items)

# Print the deciles
for i, decile in enumerate(deciles):
    print(f"Decile {i + 1}: {decile}")

Decile 1: [Item(index=2, value=10001, weight=10002, value_per_weight=0.9999000199960008), Item(index=4, value=10252, weight=10254, value_per_weight=0.9998049541642285), Item(index=6, value=10503, weight=10506, value_per_weight=0.9997144488863506)]
Decile 2: [Item(index=8, value=10754, weight=10758, value_per_weight=0.9996281836772635), Item(index=10, value=11005, weight=11010, value_per_weight=0.9995458673932789), Item(index=12, value=11256, weight=11262, value_per_weight=0.9994672349493873)]
Decile 3: [Item(index=14, value=11507, weight=11514, value_per_weight=0.9993920444676047), Item(index=16, value=11758, weight=11766, value_per_weight=0.9993200747917729), Item(index=18, value=12009, weight=12018, value_per_weight=0.9992511233150274)]
Decile 4: [Item(index=20, value=12260, weight=12270, value_per_weight=0.9991850040749797), Item(index=22, value=12511, weight=12522, value_per_weight=0.9991215460789011), Item(index=24, value=12762, weight=12774, value_per_weight=0.9990605918271489)]


In [42]:
num_lists = 10
equal_distribution = [items[i::num_lists] for i in range(num_lists)]

for i, sublist in enumerate(equal_distribution):
    print(f"List {i + 1}: {sublist}")

List 1: [Item(index=0, value=90000, weight=90001, value_per_weight=0.9999888890123443), Item(index=10, value=11005, weight=11010, value_per_weight=0.9995458673932789), Item(index=20, value=12260, weight=12270, value_per_weight=0.9991850040749797)]
List 2: [Item(index=1, value=89750, weight=89751, value_per_weight=0.9999888580628629), Item(index=11, value=88500, weight=88501, value_per_weight=0.9999887006926476), Item(index=21, value=87250, weight=87251, value_per_weight=0.9999885388133087)]
List 3: [Item(index=2, value=10001, weight=10002, value_per_weight=0.9999000199960008), Item(index=12, value=11256, weight=11262, value_per_weight=0.9994672349493873), Item(index=22, value=12511, weight=12522, value_per_weight=0.9991215460789011)]
List 4: [Item(index=3, value=89500, weight=89501, value_per_weight=0.9999888269404811), Item(index=13, value=88250, weight=88251, value_per_weight=0.999988668683641), Item(index=23, value=87000, weight=87001, value_per_weight=0.9999885058792428)]
List 5: [

In [47]:
for i in range(num_lists)
    items[i::num_lists] 

[10]